### Choose dataset

In [1]:
# DATASET_NAME = 'COMPAS_SEX'
# PROT_ATTR_IDX = 5 #COMPAS_SEX

DATASET_NAME = 'RECRUIT_SEX'
PROT_ATTR_IDX = 0 #RECRUIT_SEX 

# DATASET_NAME = 'ADULT_RACE'
# PROT_ATTR_IDX = 8 #ADULT_RACE 

# DATASET_NAME = 'SURGICAL_AGE'
# PROT_ATTR_IDX = 1 #SURGICAL_AGE

# DATASET_NAME = 'LAW_SEX'
# PROT_ATTR_IDX = 8 #LAW_SEX

In [2]:
!pip install scikit-learn-intelex

In [3]:
CTGAN_FACTOR = 5
N_AUG = 6
NFOLDS = 5
EXP = 'RF'

In [4]:
# Update numpy to a compatible version
import scipy as sp
from scipy.io import arff
import numpy as np
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
!pip install ctgan

In [6]:
!pip install sdv

In [7]:
from sdv.metadata import SingleTableMetadata

c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from sdv.single_table import CTGANSynthesizer

In [9]:
!pip install wandb --upgrade


In [10]:
import wandb

In [11]:
wandb.login(key='a6bbbea4dc56f4a415d7e588526ee9bab53ccdfb')

wandb: Currently logged in as: rabaevn (rabaevn-ben-gurion-university-of-the-negev). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\seffi\_netrc


True

In [12]:
!pip install fairlearn

In [13]:
!pip install scikit-learn==1.2.2 category_encoders

In [14]:
import pandas as pd 

import os
import math
import gc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_auc_score, confusion_matrix

warnings.simplefilter(action='ignore', category=FutureWarning)

from fairlearn.postprocessing import ThresholdOptimizer

from datetime import datetime

from sklearn.metrics import confusion_matrix
import graphviz

from sklearn.tree import export_graphviz

from datetime import date

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [15]:
if DATASET_NAME == 'ADULT_SEX':
    TARGET_COL = 'income'
    PRIV_FEATURE = "sex"
    PRIV_VALUE_FEATURE = 1 # in this dataset the target = 0 is unfavorable.
    PRIV_GROUP = 1 # male
    data = pd.read_csv('Data/adult.csv')
    data[TARGET_COL] = data[TARGET_COL].apply(lambda x: 0 if x == '<=50K' else 1)
    data['sex'] = np.where(data['sex'] == 'Male', 0,1)
    data['white'] = np.where(data['race'] == 'White', 1,0)


    feature_columns = ['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country', 'white']

    CATEGORICAL = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'native.country']
    #CTGAN
    metadata1 = SingleTableMetadata()
    metadata1.detect_from_dataframe(data=data[feature_columns])
    ctgan1 = CTGANSynthesizer(metadata1, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan1.fit(data[feature_columns])
    ctgan1.save('ctgan_adult_sex.pkl')


elif DATASET_NAME == 'CREDIT_SEX':
    data = pd.read_csv('Data/german_credit_data.csv')
    TARGET_COL = 'Risk'
    PRIV_FEATURE = 'Sex'
    PRIV_GROUP = 1
    # in this dataset the target = 1 is unfavorable.
    PRIV_VALUE_FEATURE = 0
    data[TARGET_COL] = data[TARGET_COL].apply(lambda x: 0 if x == 'bad' else 1)
    data['Sex'] = np.where(data['Sex'] == 'male', 0,1)
    data = data.drop(["Unnamed: 0"],axis=1)
    data['young'] = data['Age'].apply(lambda x: 0 if x < 25 else 1)
    CATEGORICAL = ['Housing', 'Saving accounts' , 'Checking account', 'Purpose']
    feature_columns = ['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account','Credit amount', 'Duration', 'Purpose']
    #N_AUG = 2
    #CTGAN
    metadata2 = SingleTableMetadata()
    metadata2.detect_from_dataframe(data=data[feature_columns])
    ctgan2 = CTGANSynthesizer(metadata2, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan2.fit(data[feature_columns])
    ctgan2.save('ctgan_credit_sex.pkl')

elif DATASET_NAME == 'RECRUIT_SEX':
    data = pd.read_csv('Data/recruitmentdataset-2022-1.3.csv')
    TARGET_COL = 'decision'
    PRIV_FEATURE = 'gender'
    PRIV_GROUP = 0
    # in this dataset the target = 0 is unfavorable.
    PRIV_VALUE_FEATURE = 1
    data['gender'] = np.where(data['gender'] == 'male', 0,1)
    data['decision'] = np.where(data['decision'] == False, 0,1)
    CATEGORICAL = [ 'nationality', 'sport',
       'ind-debateclub', 'ind-programming_exp', 'ind-international_exp',
       'ind-entrepeneur_exp', 'ind-exact_study', 'ind-degree',
       'company']
    feature_columns = ['gender', 'age', 'nationality', 'sport', 'ind-university_grade',
       'ind-debateclub', 'ind-programming_exp', 'ind-international_exp',
       'ind-entrepeneur_exp', 'ind-languages', 'ind-exact_study', 'ind-degree',
       'company']

    #CTGAN
    metadata3 = SingleTableMetadata()
    metadata3.detect_from_dataframe(data=data[feature_columns])
    ctgan3 = CTGANSynthesizer(metadata3, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan3.fit(data[feature_columns])
    ctgan3.save('ctgan_recruit_sex.pkl')

elif DATASET_NAME =='COMPAS_SEX':
    data = pd.read_csv('Data/compas-scores-two-years_v1.csv')
    TARGET_COL = 'two_year_recid'
    data['sex'] = np.where(data['sex'] == 'Male', 0,1)
    PRIV_FEATURE = 'sex'
    PRIV_GROUP = 1
    # in this dataset the target = 1 is unfavorable.
    PRIV_VALUE_FEATURE = 0
    feature_columns = ['sex', 'age', 'age_cat', 'race', 'juv_fel_count', 'juv_misd_count', 'priors_count', 'c_charge_degree' , 'score_text', 'v_score_text']
    CATEGORICAL = [ 'age_cat', 'race', 'c_charge_degree', 'score_text', 'v_score_text']
    #CTGAN
    metadata4 = SingleTableMetadata()
    metadata4.detect_from_dataframe(data=data[feature_columns])
    ctgan4 = CTGANSynthesizer(metadata4, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan4.fit(data[feature_columns])
    ctgan4.save('ctgan_compas_sex.pkl')

elif DATASET_NAME == 'DIABETES_AGE':
    data = pd.read_csv('Data/diabetes_prediction_dataset.csv')
    TARGET_COL = 'diabetes'
    data['age_binary'] = data['age'].apply(lambda x: 0 if x < 40 else 1)
    PRIV_FEATURE = 'age_binary'
    PRIV_GROUP = 1
    # in this dataset the target = 1 is unfavorable.
    PRIV_VALUE_FEATURE = 0
    feature_columns = ['gender', 'age_binary', 'hypertension', 'heart_disease', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level']
    CATEGORICAL = ['smoking_history', 'gender']
    #CTGAN
    metadata5 = SingleTableMetadata()
    metadata5.detect_from_dataframe(data=data[feature_columns])
    ctgan5 = CTGANSynthesizer(metadata5, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan5.fit(data[feature_columns])
    ctgan5.save('ctgan_diabetes_age.pkl')

elif DATASET_NAME == 'BANK_AGE':
    data = pd.read_csv('Data/bank.csv')
    data['age'] = np.where(data['age'] < 40, 0, 1)
    data['deposit'] = np.where(data['deposit'] == "no", 0,1)
    TARGET_COL = 'deposit'
    PRIV_FEATURE = 'age'
    PRIV_GROUP = 0
    # in this dataset target = 0 is unfavorable
    PRIV_VALUE_FEATURE = 1
    feature_columns = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
    CATEGORICAL = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
    #CTGAN
    metadata6 = SingleTableMetadata()
    metadata6.detect_from_dataframe(data=data[feature_columns])
    ctgan6 = CTGANSynthesizer(metadata6, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan6.fit(data[feature_columns])
    ctgan6.save('ctgan_bank_age.pkl')


elif DATASET_NAME == 'DEFAULT_CREDIT_SEX':
    data = pd.read_csv('Data/UCI_Credit_Card.csv')
    data['SEX'] = np.where(data['SEX'] == 1, 0, 1) # male - 0, female - 1
    TARGET_COL = 'default.payment.next.month'
    PRIV_FEATURE = 'SEX'
    PRIV_GROUP = 0
    # in this dataset target = 1 is unfavorable
    PRIV_VALUE_FEATURE = 0
    feature_columns = ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE",
    "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6",
    "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6",
    "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]
    CATEGORICAL = []
    #CTGAN
    metadata7 = SingleTableMetadata()
    metadata7.detect_from_dataframe(data=data[feature_columns])
    ctgan7 = CTGANSynthesizer(metadata7, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan7.fit(data[feature_columns])
    ctgan7.save('ctgan_default_credit_sex.pkl')

elif DATASET_NAME == 'NURSERY_OCC':
    data = pd.read_csv('Data/nursery.csv')
    data['parents'] = np.where(data['parents'] == 'great_pret', 0, 1)
    data['final evaluation'] = np.where(data['final evaluation'] == 'not_recom', 0, 1)
    data['children'] = np.where(data['children'] == "more", 4, data['children'])
    TARGET_COL = 'final evaluation'
    PRIV_FEATURE = 'parents'
    PRIV_GROUP = 0
    # in this dataset target = 0 is unfavorable
    PRIV_VALUE_FEATURE = 1
    feature_columns = ["parents", "has_nurs", "form", "children", "housing", "finance", "social", "health"] # we dont use health becuase it has high correlation with the target
    CATEGORICAL = ["has_nurs", "form", "housing", "finance", "social", "health"]
    #CTGAN
    metadata8 = SingleTableMetadata()
    metadata8.detect_from_dataframe(data=data[feature_columns])
    ctgan8 = CTGANSynthesizer(metadata8, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan8.fit(data[feature_columns])
    ctgan8.save('ctgan_nursery_occ.pkl')

elif DATASET_NAME == 'COMPAS_RACE':
    data = pd.read_csv('Data/compas-scores-two-years_v1.csv')
    TARGET_COL = 'two_year_recid'
    data['race'] = np.where(data['race'] == 'African-American', 0,1)
    PRIV_FEATURE = 'race'
    PRIV_GROUP = 1
    # in this dataset the target = 1 is unfavorable.
    PRIV_VALUE_FEATURE = 0
    feature_columns = ['sex', 'age', 'age_cat', 'race', 'juv_fel_count', 'juv_misd_count', 'priors_count', 'c_charge_degree' , 'score_text', 'v_score_text']
    CATEGORICAL = ['sex', 'age_cat', 'c_charge_degree', 'score_text', 'v_score_text']
    #CTGAN
    metadata9 = SingleTableMetadata()
    metadata9.detect_from_dataframe(data=data[feature_columns])
    ctgan9 = CTGANSynthesizer(metadata9, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan9.fit(data[feature_columns])
    ctgan9.save('ctgan_compas_race.pkl')

elif DATASET_NAME == 'ADULT_RACE':
    TARGET_COL = 'income'
    PRIV_FEATURE = 'race'
    PRIV_VALUE_FEATURE = 1 # in this dataset the target = 0 is unfavorable.
    PRIV_GROUP = 1 # white
    data = pd.read_csv('Data/adult.csv')
    data[TARGET_COL] = data[TARGET_COL].apply(lambda x: 0 if x == '<=50K' else 1)
    data['race'] = np.where(data['race'] == 'Black', 0,1)
    data['white'] = np.where(data['race'] == 'White', 1,0)
    feature_columns = ['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country', 'white']

    CATEGORICAL = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'sex', 'native.country']
    #CTGAN
    metadata10 = SingleTableMetadata()
    metadata10.detect_from_dataframe(data=data[feature_columns])
    ctgan10 = CTGANSynthesizer(metadata10, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan10.fit(data[feature_columns])
    ctgan10.save('ctgan_adult_race.pkl')

elif DATASET_NAME == 'MIMIC_SEX':
    TARGET_COL = 'outcome'
    PRIV_FEATURE = 'gendera'
    PRIV_VALUE_FEATURE = 1 # in this tadaset target = 0 is unfavorable
    PRIV_GROUP = 1 # male
    data = pd.read_csv('Data/MIMIC2.csv')
    data['gendera'] = np.where(data['gendera'] == 2, 0, 1)
    feature_columns = ['group', 'age', 'gendera', 'BMI', 'hypertensive',
       'atrialfibrillation', 'CHD with no MI', 'diabetes', 'deficiencyanemias',
       'depression', 'Hyperlipemia', 'Re-999l failure', 'COPD', 'heart rate',
       'Systolic blood pressure', 'Diastolic blood pressure',
       'Respiratory rate', 'temperature', 'SP O2', 'Urine output',
       'hematocrit', 'RBC', 'MCH', 'MCHC', 'MCV', 'RDW', 'Leucocyte',
       'Platelets', 'Neutrophils', 'Basophils', 'Lymphocyte', 'PT', 'INR',
       'NT-proBNP', 'Creatine ki-999se', 'Creatinine', 'Urea nitrogen',
       'glucose', 'Blood potassium', 'Blood sodium', 'Blood calcium',
       'Chloride', 'Anion gap', 'Magnesium ion', 'PH', 'Bicarbo-999te',
       'Lactic acid', 'PCO2', 'EF']
    # if not working remove gendera from categorical
    CATEGORICAL =  [
    'group', 'hypertensive', 'atrialfibrillation',
    'CHD with no MI', 'diabetes', 'deficiencyanemias', 'depression',
    'Hyperlipemia', 'Re-999l failure', 'COPD'
    ]

    #CTGAN
    metadata11 = SingleTableMetadata()
    metadata11.detect_from_dataframe(data=data[feature_columns])
    ctgan11 = CTGANSynthesizer(metadata11, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan11.fit(data[feature_columns])
    ctgan11.save('ctgan_mimic_sex.pkl')

elif DATASET_NAME == 'MATERNAL_AGE':
    TARGET_COL = 'RiskLevel'
    PRIV_FEATURE = 'Age'
    PRIV_VALUE_FEATURE = 0 #in this dataset target = 1 is unfavorable
    PRIV_GROUP = 0 #young
    data = pd.read_csv('Data/MaternalBinary.csv')
    data['Age'] = np.where(data['Age'] < 40, 0, 1)
    feature_columns = ['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate']
    CATEGORICAL = []

    #CTGAN
    metadata12 = SingleTableMetadata()
    metadata12.detect_from_dataframe(data=data[feature_columns])
    ctgan12 = CTGANSynthesizer(metadata12, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan12.fit(data[feature_columns])
    ctgan12.save('ctgan_maternal_age.pkl')

elif DATASET_NAME == 'MESO_SEX':
    TARGET_COL = 'class of diagnosis'
    PRIV_FEATURE = 'gender'
    PRIV_VALUE_FEATURE = 0 # in this dataset, target = 1 is unfavorable
    PRIV_GROUP = 1 #male
    data = pd.read_csv('Data/Mesothelioma.csv')
    feature_columns = ['age', 'gender', 'city', 'asbestos exposure', 'type of MM', 'duration of asbestos exposure',
                       'keep side', 'cytology', 'duration of symptoms', 'dyspnoea', 'ache on chest', 'weakness',
                       'habit of cigarette', 'performance status', 'white blood', 'cell count (WBC)', 'hemoglobin (HGB)',
                       'platelet count (PLT)', 'sedimentation', 'blood lactic dehydrogenise (LDH)', 'alkaline phosphatise (ALP)',
                       'total protein', 'albumin', 'glucose', 'pleural lactic dehydrogenise', 'pleural protein',
                       'pleural albumin', 'pleural glucose', 'dead or not', 'pleural effusion', 'pleural thickness on tomography',
                       'pleural level of acidity (pH)', 'C-reactive protein (CRP)']
    CATEGORICAL = ['city', 'asbestos exposure', 'type of MM', 'keep side', 'cytology', 'dyspnoea', 'ache on chest',
                   'weakness', 'habit of cigarette', 'performance status', 'hemoglobin (HGB)', 'dead or not', 'pleural effusion',
                   'pleural thickness on tomography', 'pleural level of acidity (pH)']
    #CTGAN
    metadata13 = SingleTableMetadata()
    metadata13.detect_from_dataframe(data=data[feature_columns])
    ctgan13 = CTGANSynthesizer(metadata13, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan13.fit(data[feature_columns])
    ctgan13.save('ctgan_meso_sex.pkl')

elif DATASET_NAME == 'SURGICAL_SEX':
    TARGET_COL = 'complication'
    PRIV_FEATURE = 'gender'
    PRIV_VALUE_FEATURE = 0 #in this dataset, target = 1 is unfavorable
    data = pd.read_csv('Data/Kaggle_Surgical-deepnet.csv')
    PRIV_GROUP = 1 # females
    feature_columns = ['bmi', 'Age', 'asa_status', 'baseline_cancer', 'baseline_charlson', 'baseline_cvd', 'baseline_dementia',
                       'baseline_diabetes', 'baseline_digestive', 'baseline_osteoart', 'baseline_psych', 'baseline_pulmonary',
                       'ahrq_ccs', 'ccsComplicationRate', 'ccsMort30Rate', 'complication_rsi', 'dow', 'gender', 'hour', 'month',
                       'moonphase', 'mort30', 'mortality_rsi', 'race']
    CATEGORICAL = ['asa_status', 'baseline_cancer', 'baseline_charlson', 'baseline_cvd', 'baseline_dementia',
                       'baseline_diabetes', 'baseline_digestive', 'baseline_osteoart', 'baseline_psych', 'baseline_pulmonary',
                     'month','dow','moonphase', 'mort30','race']
        #CTGAN
    metadata14 = SingleTableMetadata()
    metadata14.detect_from_dataframe(data=data[feature_columns])
    ctgan14 = CTGANSynthesizer(metadata14, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan14.fit(data[feature_columns])
    ctgan14.save('ctgan_surgical_sex.pkl')
elif DATASET_NAME == 'SURGICAL_AGE':
    TARGET_COL = 'complication'
    PRIV_FEATURE = 'Age'
    PRIV_VALUE_FEATURE = 0 #in this dataset, target = 1 is unfavorable
    data = pd.read_csv('Data/Kaggle_Surgical-deepnet.csv')
    data['Age'] = np.where(data['Age'] < 65, 0, 1)
    PRIV_GROUP = 1 # young
    feature_columns = ['bmi', 'Age', 'asa_status', 'baseline_cancer', 'baseline_charlson', 'baseline_cvd', 'baseline_dementia',
                       'baseline_diabetes', 'baseline_digestive', 'baseline_osteoart', 'baseline_psych', 'baseline_pulmonary',
                       'ahrq_ccs', 'ccsComplicationRate', 'ccsMort30Rate', 'complication_rsi', 'dow', 'gender', 'hour', 'month',
                       'moonphase', 'mort30', 'mortality_rsi', 'race']
    CATEGORICAL = ['asa_status','gender', 'baseline_cancer', 'baseline_charlson', 'baseline_cvd', 'baseline_dementia',
                       'baseline_diabetes', 'baseline_digestive', 'baseline_osteoart', 'baseline_psych', 'baseline_pulmonary',
                     'month','dow','moonphase', 'mort30','race']
        #CTGAN
    metadata15 = SingleTableMetadata()
    metadata15.detect_from_dataframe(data=data[feature_columns])
    ctgan15 = CTGANSynthesizer(metadata15, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan15.fit(data[feature_columns])
    ctgan15.save('ctgan_surgical_age.pkl')

elif DATASET_NAME == 'DUTCH_SEX':
    data = pd.read_csv('Data/dutch_census_2001.csv')
    TARGET_COL = 'occupation'
    PRIV_FEATURE = 'sex'
    PRIV_VALUE_FEATURE = 1 # in this dataset, target=0 is unfavorable
    data['sex'] = np.where(data['sex'] == 1, 0, 1) # convert 0 to males and 1 to females
    data['occupation'] = np.where(data['occupation'] == '2_1', 0, 1)
    PRIV_GROUP = 0 # male
    feature_columns = ['sex', 'age', 'household_position', 'household_size', 'prev_residence_place', 'citizenship', 'country_birth',
                       'edu_level', 'economic_status', 'cur_eco_activity', 'Marital_status']
    CATEGORICAL = ['age', 'household_position', 'household_size', 'citizenship', 'country_birth',
                  'edu_level', 'economic_status', 'cur_eco_activity', 'Marital_status']


    # CTGAN
    metadata16 = SingleTableMetadata()
    metadata16.detect_from_dataframe(data=data[feature_columns])
    ctgan16 = CTGANSynthesizer(metadata16, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan16.fit(data[feature_columns])
    ctgan16.save('ctgan_dutch_sex.pkl')

elif DATASET_NAME == 'LAW_SEX':
    data = pd.read_csv('Data/law_school_clean.csv')
    TARGET_COL = 'pass_bar'
    PRIV_FEATURE = 'male'
    PRIV_VALUE_FEATURE = 1 # in this dataset, target = 0 is unfavorable
    PRIV_GROUP = 1 # male
    feature_columns = ['decile1b', 'decile3', 'lsat', 'ugpa', 'zfygpa', 'zgpa', 'fulltime', 'fam_inc', 'male', 'tier', 'race']
    CATEGORICAL = ['fam_inc', 'tier', 'race']
    # CTGAN
    metadata17 = SingleTableMetadata()
    metadata17.detect_from_dataframe(data=data[feature_columns])
    ctgan17 = CTGANSynthesizer(metadata17, verbose=True, epochs=100 ,embedding_dim=32,generator_dim=(256,128,64,32),discriminator_dim=(256,128,64,32), discriminator_lr=5e-6,generator_lr=5e-6)

    ctgan17.fit(data[feature_columns])
    ctgan17.save('ctgan_law_sex.pkl')

c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Gen. (1.12) | Discrim. (-0.02): 100%|██████████| 100/100 [00:48<00:00,  2.07it/s]


### prepare datasets

# PreProcessing

In [16]:
# Feature Selection

#dataset for adverserial validaiton of the privileged feature
priv_feature_columns = feature_columns.copy()
#priv_feature_columns.append(TARGET_COL)
priv_feature_columns.remove(PRIV_FEATURE)

priv_data = data[priv_feature_columns].copy()
priv_y = data[PRIV_FEATURE].copy()

# dataset for training
y = data[TARGET_COL]
data = data[feature_columns]

import category_encoders as ce
encoder = ce.TargetEncoder(cols=CATEGORICAL)

encoder.fit(data, y)
data = encoder.transform(data)



encoder1 = ce.TargetEncoder(cols=CATEGORICAL)
encoder1.fit(priv_data, priv_y)
priv_data = encoder1.transform(priv_data)



In [17]:
def print_metrics(y_gt, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_gt, y_pred).ravel()
    print('conf matrix:\n ', confusion_matrix(y_gt, y_pred) ,
          '\naccuracy: ', (tp+tn)/(tp+tn+fn+fp),
          'precision: ', tp/(tp+fp),
          'recall: ', tp/(tp+fn),
          'fpr: ', fp/(fp+tn),
          'tpr: ', tp /(tp + fn),
          'fn+fp', fn+fp)
    #returm fpr, tpr, accuracy
    return fp/(fp+tn), tp /(tp + fn), (tp+tn)/(tp+tn+fn+fp)


In [18]:
# bias metric - equal opportunity difference
def opportunity_diff_tpr(tpr1, tpr2):
    return abs(tpr1-tpr2)

In [19]:
# bias metric - equal opportunity difference
def opportunity_diff_fpr(fpr1, fpr2):
    return abs(fpr1-fpr2)

In [20]:
# bias metric - avarage absolute odds difference
def odds_diff(tpr1, tpr2, fpr1, fpr2):
    return 0.5*(abs(tpr2-tpr1) + abs(fpr2-fpr1))

In [21]:
# bias metric - statistical parity difference
def parity_diff(y_pred_0, y_pred_1):
    if PRIV_GROUP ==1:
        return len(y_pred_0)/(len(y_pred_0)+len(y_pred_1)) - len(y_pred_1)/(len(y_pred_1)+len(y_pred_0))
    else:
        return len(y_pred_1)/(len(y_pred_0)+len(y_pred_1)) - len(y_pred_0)/(len(y_pred_1)+len(y_pred_0))

In [22]:
# bias metric - Disparate impact
def disparate_impact(y_pred_0, y_pred_1):
    if PRIV_VALUE_FEATURE == 1:
        rate_0 = sum(y_pred_0) / len(y_pred_0) if len(y_pred_0) > 0 else 0
        rate_1 = sum(y_pred_1) / len(y_pred_1) if len(y_pred_1) > 0 else 0
    else:
        rate_0 = sum(1 - y for y in y_pred_0) / len(y_pred_0) if len(y_pred_0) > 0 else 0
        rate_1 = sum(1 - y for y in y_pred_1) / len(y_pred_1) if len(y_pred_1) > 0 else 0

    if PRIV_GROUP == 1:
        return rate_0 / rate_1 if rate_1 > 0 else 0
    else:
        return rate_1 / rate_0 if rate_0 > 0 else 0


In [23]:
# calculate the bias of divided dataset by the privileged feature
def calc_bias(data,y, priv_feature,y_pred, experiment_text):
    data_full =  pd.concat([data, y], axis=1)
    data_full['y_pred'] = y_pred

    label = TARGET_COL
    print('performance for whole ds:'+ experiment_text)
    fpr, tpr, acc = print_metrics(data_full[TARGET_COL], data_full['y_pred'] )

    feature_cat_list = list(data[priv_feature].unique())
    #print(feature_cat_list)
    valid_cat0 = data_full[data_full[priv_feature]==feature_cat_list[0]]
    valid_cat1 = data_full[data_full[priv_feature]==feature_cat_list[1]]
    print('performance for 0 sub-group:')
    fpr_0, tpr_0 , acc_0 = print_metrics(valid_cat0[TARGET_COL], valid_cat0['y_pred'])
    print('performance for 1 sub-group:')

    fpr_1, tpr_1, acc_1 = print_metrics(valid_cat1[TARGET_COL], valid_cat1['y_pred'])
    op_diff = opportunity_diff_tpr(tpr_0, tpr_1)
    op_diff_fpr = opportunity_diff_fpr(fpr_0, fpr_1)
    od_diff = odds_diff(tpr_0, tpr_1, fpr_0, fpr_1)
    print('bias metrics:')

    print('avarage absolute odds difference: (close to 0)',od_diff)
    di = disparate_impact(valid_cat0['y_pred'].astype(int),valid_cat1['y_pred'].astype(int))

    return (op_diff,op_diff_fpr, od_diff, acc,fpr_0, tpr_0, fpr_1, tpr_1, di)


# TTA implementation

# Modeling

## General baseline model

In [24]:

cv = StratifiedKFold(n_splits=NFOLDS, random_state=1234, shuffle=True)
results = pd.DataFrame(columns=['training_score', 'test_score'])

if EXP =='RF':
    clf = RandomForestClassifier(random_state=0, min_samples_leaf = 7)#max_depth=50, , min_samples_leaf = 7
elif EXP == 'XGB':
    clf = XGBClassifier(max_depth=5, n_estimators=40, subsample=0.9, learning_rate=0.1, random_state = 142)
elif EXP == 'NN':
    clf = MLPClassifier(hidden_layer_sizes = 157, solver = 'adam', learning_rate = 'adaptive', random_state = 142)

else:
    clf = LogisticRegression(random_state = 142, n_jobs = -1)

fprs, tprs, scores, op,op_f,  od , acc , fpr_0, tpr_0, fpr_1, tpr_1, d_i = [], [], [], [],[], [], [], [], [], [], [], []

start_time = datetime.now()
for (train, test), i in zip(cv.split(data, y), range(5)):
    clf.fit(data.iloc[train], y.iloc[train])
    #_, _, auc_score_train = compute_roc_auc(train)
    #fpr, tpr, auc_score = compute_roc_auc(test)
    #scores.append((auc_score_train, auc_score))
    # fprs.append(fpr)
    # tprs.append(tpr)
    y_pred = clf.predict(data.iloc[test])
    o_p, o_p_fpr, o_d , acc_ , fpr_0_, tpr_0_, fpr_1_, tpr_1_, d_i_ = calc_bias(data.iloc[test],  y.iloc[test], PRIV_FEATURE,y_pred, 'subexperiment - baseline wo/TTA'+' model:'+str(i))
    acc.append(acc_)
    op.append(o_p)
    op_f.append(o_p_fpr)
    od.append(o_d)
    fpr_0.append(fpr_0_)
    tpr_0.append(tpr_0_)
    fpr_1.append(fpr_1_)
    tpr_1.append(tpr_1_)
    d_i.append(d_i_)
time_elapsed_baseline = datetime.now() - start_time


performance for whole ds:subexperiment - baseline wo/TTA model:0
conf matrix:
  [[511  35]
 [ 82 172]] 
accuracy:  0.85375 precision:  0.8309178743961353 recall:  0.6771653543307087 fpr:  0.0641025641025641 tpr:  0.6771653543307087 fn+fp 117
performance for 0 sub-group:
conf matrix:
  [[226  28]
 [ 43 108]] 
accuracy:  0.8246913580246914 precision:  0.7941176470588235 recall:  0.7152317880794702 fpr:  0.11023622047244094 tpr:  0.7152317880794702 fn+fp 71
performance for 1 sub-group:
conf matrix:
  [[285   7]
 [ 39  64]] 
accuracy:  0.8835443037974684 precision:  0.9014084507042254 recall:  0.6213592233009708 fpr:  0.023972602739726026 tpr:  0.6213592233009708 fn+fp 46
bias metrics:
avarage absolute odds difference: (close to 0) 0.09006809125560714
performance for whole ds:subexperiment - baseline wo/TTA model:1
conf matrix:
  [[504  42]
 [ 77 177]] 
accuracy:  0.85125 precision:  0.8082191780821918 recall:  0.6968503937007874 fpr:  0.07692307692307693 tpr:  0.6968503937007874 fn+fp 119

In [25]:
print('FPR:' , np.mean(fpr_0),np.mean(fpr_1) )
print('TPR:', np.mean(tpr_0), np.mean(tpr_1))
print('beacause of tpr:')
if np.mean(tpr_1) > np.mean(tpr_0):
    print ('group z = 0 experiences more False Negatives relative to class 0')
else:
    print ('group z = 1 experiences more False Negatives relative to class 0')
print('because of fpr:')
if np.mean(fpr_1) > np.mean(fpr_0):
    print('group z = 1 experiences more False Positives relative to class 1')
else:
    print('group z = 0 experiences more False Positives relative to class 1')


FPR: 0.07582222154569669 0.07545676884475949
TPR: 0.675039073023873 0.70387420816419
beacause of tpr:
group z = 0 experiences more False Negatives relative to class 0
because of fpr:
group z = 0 experiences more False Positives relative to class 1


## Baseline using Reject Option

In [26]:
import numpy as np
from sklearn.metrics import confusion_matrix


def apply_reject_option_classification(predictions, protected_attribute, threshold=0.5, uncertainty_range=0.05):


    """
    Apply the reject option to mitigate bias in model predictions.
    Parameters:
    - predictions: array-like, model predictions (probabilities) before applying the reject option.
    - protected_attribute: array-like, the protected attribute based on which bias might occur (0 or 1).
    - threshold: float, the decision threshold for classification.
    - uncertainty_range: float, defines the region of uncertainty around the threshold.

    Returns:
    - corrected_predictions: array, the predictions after applying the reject option for bias mitigation.
    """
    global flip_counter

    # Initialize corrected predictions with the original predictions
    corrected_predictions = np.zeros(predictions.shape)
    decision_boundary_lower = threshold - uncertainty_range
    decision_boundary_upper = threshold + uncertainty_range
    for i, pred in enumerate(predictions):
          # Check if prediction falls within the region of uncertainty
          if decision_boundary_lower < pred < decision_boundary_upper:
              # Apply mitigation strategy: Here, we flip the decision if it correlates with the protected attribute
              # This is a simple strategy and can be replaced with more sophisticated ones
              if protected_attribute[i] != PRIV_GROUP:
                  if predictions[i] != PRIV_VALUE_FEATURE:
                    corrected_predictions[i] = 1 - (pred > threshold) # change prediction
                    flip_counter += 1
                  else: # dont change prediction
                    corrected_predictions[i] = pred > threshold
              else:
                  corrected_predictions[i] = pred > threshold
          else:
              # Outside the region of uncertainty, keep the original decision
              corrected_predictions[i] = pred > threshold
    return corrected_predictions


cv = StratifiedKFold(n_splits=NFOLDS, random_state=1234, shuffle=True)
results = pd.DataFrame(columns=['training_score', 'test_score'])

if EXP =='RF':
    clf = RandomForestClassifier(random_state=0, min_samples_leaf = 7)
elif EXP == 'XGB':
    clf = XGBClassifier(max_depth=5, n_estimators=40, subsample=0.9, learning_rate=0.1, random_state = 142)
elif EXP == 'NN':
    clf = MLPClassifier(hidden_layer_sizes = 157, solver = 'adam', learning_rate = 'adaptive', random_state = 142)
else:
    clf = LogisticRegression(random_state = 142)

fprs, tprs, scores, op_tho,op_f_tho, od_tho, acc_tho, fpr_0_tho, tpr_0_tho, fpr_1_tho, tpr_1_tho, d_i_tho = [], [], [], [],[], [], [], [], [], [], [], []
flip_counter = 0
start_time = datetime.now()
for (train, test), i in zip(cv.split(data, y), range(5)):
    clf.fit(data.iloc[train], y.iloc[train].astype(int))

    y_pred_tho = []
    reg_preds = clf.predict_proba(data.iloc[test])
    y_pred_tho = apply_reject_option_classification(reg_preds[:,1], data.iloc[test][PRIV_FEATURE].values)
    o_p_tho,o_p_tho_fpr, o_d_tho, acc_tho_, fpr_0_tho_, tpr_0_tho_, fpr_1_tho_, tpr_1_tho_, d_i_tho_ = calc_bias(data.iloc[test],  y.iloc[test], PRIV_FEATURE,y_pred_tho, 'subexperiment - baseline THO'+' model:'+str(i))
    op_tho.append(o_p_tho)
    op_f_tho.append(o_p_tho_fpr)
    od_tho.append(o_d_tho)
    acc_tho.append(acc_tho_)

    fpr_0_tho.append(fpr_0_tho_)
    tpr_0_tho.append(tpr_0_tho_)
    fpr_1_tho.append(fpr_1_tho_)
    tpr_1_tho.append(tpr_1_tho_)
    d_i_tho.append(d_i_tho_)
time_elapsed_tho = datetime.now() - start_time



performance for whole ds:subexperiment - baseline THO model:0
conf matrix:
  [[505  41]
 [ 83 171]] 
accuracy:  0.845 precision:  0.8066037735849056 recall:  0.6732283464566929 fpr:  0.07509157509157509 tpr:  0.6732283464566929 fn+fp 124
performance for 0 sub-group:
conf matrix:
  [[226  28]
 [ 43 108]] 
accuracy:  0.8246913580246914 precision:  0.7941176470588235 recall:  0.7152317880794702 fpr:  0.11023622047244094 tpr:  0.7152317880794702 fn+fp 71
performance for 1 sub-group:
conf matrix:
  [[279  13]
 [ 40  63]] 
accuracy:  0.8658227848101265 precision:  0.8289473684210527 recall:  0.6116504854368932 fpr:  0.04452054794520548 tpr:  0.6116504854368932 fn+fp 53
bias metrics:
avarage absolute odds difference: (close to 0) 0.08464848758490626
performance for whole ds:subexperiment - baseline THO model:1
conf matrix:
  [[501  45]
 [ 71 183]] 
accuracy:  0.855 precision:  0.8026315789473685 recall:  0.7204724409448819 fpr:  0.08241758241758242 tpr:  0.7204724409448819 fn+fp 116
performan

c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


performance for whole ds:subexperiment - baseline THO model:3
conf matrix:
  [[501  46]
 [ 75 178]] 
accuracy:  0.84875 precision:  0.7946428571428571 recall:  0.7035573122529645 fpr:  0.08409506398537477 tpr:  0.7035573122529645 fn+fp 121
performance for 0 sub-group:
conf matrix:
  [[253  26]
 [ 32  72]] 
accuracy:  0.8485639686684073 precision:  0.7346938775510204 recall:  0.6923076923076923 fpr:  0.0931899641577061 tpr:  0.6923076923076923 fn+fp 58
performance for 1 sub-group:
conf matrix:
  [[248  20]
 [ 43 106]] 
accuracy:  0.8489208633093526 precision:  0.8412698412698413 recall:  0.7114093959731543 fpr:  0.07462686567164178 tpr:  0.7114093959731543 fn+fp 63
bias metrics:
avarage absolute odds difference: (close to 0) 0.01883240107576318
performance for whole ds:subexperiment - baseline THO model:4
conf matrix:
  [[497  50]
 [ 76 177]] 
accuracy:  0.8425 precision:  0.7797356828193832 recall:  0.6996047430830039 fpr:  0.09140767824497258 tpr:  0.6996047430830039 fn+fp 126
perform

In [27]:
experiment = 'baseline with Reject Option'
run = wandb.init(project='BiasGuardRF1', reinit = True)
print(experiment)
print('tpr0: ', np.mean(tpr_0_tho) , ' tpr1: ', np.mean(tpr_1_tho), ' delta: ', abs(np.mean(tpr_0_tho) - np.mean(tpr_1_tho)))
print('fpr0: ', np.mean(fpr_0_tho), 'fpr1: ', np.mean(fpr_1_tho), ' delta: ', abs(np.mean(fpr_0_tho) - np.mean(fpr_1_tho)))
print('equal opportunity for tpr: ',np.mean(op_tho))
print('equal opportunity for fpr: ',np.mean(op_f_tho))
print('avarage absolute odds difference: ',np.mean(od_tho))
print('accuracy: ', np.mean(acc_tho))
print('Time: ',time_elapsed_tho)
wandb.log({'dataset' : DATASET_NAME, 'model' : EXP, 'experiment' : experiment,
            'accuracy' : np.mean(acc_tho), 'acc_var': np.var(acc_tho), 'dTPR' : np.mean(op_tho), 'dFPR':np.mean(op_f_tho),
            'avarge odd' : np.mean(od_tho), 'odd_var': np.var(od_tho), 'DI':np.mean(d_i_tho), 'DI_var': np.var(d_i_tho), 'flips': flip_counter, 'time' : str(time_elapsed_tho) }, step=0)
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


baseline with Reject Option
tpr0:  0.6973104909263589  tpr1:  0.6998271974334798  delta:  0.0025167065071208805
fpr0:  0.08602313709085876 fpr1:  0.08032107486698746  delta:  0.005702062223871304
equal opportunity for tpr:  0.0942623521477625
equal opportunity for fpr:  0.03451821106837488
avarage absolute odds difference:  0.0643902816080687
accuracy:  0.8504999999999999
Time:  0:00:00.258315


DI,▁
DI_var,▁
acc_var,▁
accuracy,▁
avarge odd,▁
dFPR,▁
dTPR,▁
flips,▁
odd_var,▁
DI,1.03601
DI_var,0.12182


## Baseline using Thresholds optimizer

In [28]:
cv = StratifiedKFold(n_splits=NFOLDS, random_state=1234, shuffle=True)
results = pd.DataFrame(columns=['training_score', 'test_score'])

if EXP =='RF':
    clf = RandomForestClassifier(random_state=0, min_samples_leaf = 7)
elif EXP == 'XGB':
    clf = XGBClassifier(max_depth=5, n_estimators=40, subsample=0.9, learning_rate=0.1, random_state = 142)
elif EXP == 'NN':
    clf = MLPClassifier(hidden_layer_sizes = 157, solver = 'adam', learning_rate = 'adaptive', random_state = 142)
else:
    clf = LogisticRegression(random_state = 142)

flips = 0
fprs, tprs, scores, op_tho,op_f_tho, od_tho, acc_tho, fpr_0_tho, tpr_0_tho, fpr_1_tho, tpr_1_tho, d_i_tho = [], [], [], [],[], [], [], [], [], [], [], []
start_time = datetime.now()
for (train, test), i in zip(cv.split(data, y), range(5)):
    clf.fit(data.iloc[train], y.iloc[train].astype(int))

    y_pred_tho = []
    pre_optimized_preds = clf.predict_proba(data.iloc[test])[:, 1]
    pre_optimized_preds_binary = (pre_optimized_preds > 0.5).astype(int) ### הוספתי את השורה הזאת
    postprocess_est = ThresholdOptimizer(
                       estimator=clf,
                       predict_method='predict_proba')
    postprocess_est.fit(data.iloc[train],  y.iloc[train].astype(int), sensitive_features=data.iloc[train][PRIV_FEATURE])
    y_pred_tho = postprocess_est.predict(data.iloc[test], sensitive_features=data.iloc[test][PRIV_FEATURE])
    flips += sum(pre_optimized_preds_binary != y_pred_tho) ### ופה שיניתי את הספירה של הפליפים
    o_p_tho,o_p_tho_fpr, o_d_tho, acc_tho_, fpr_0_tho_, tpr_0_tho_, fpr_1_tho_, tpr_1_tho_, d_i_tho_ = calc_bias(data.iloc[test],  y.iloc[test], PRIV_FEATURE,y_pred_tho, 'subexperiment - baseline THO'+' model:'+str(i))
    op_tho.append(o_p_tho)
    op_f_tho.append(o_p_tho_fpr)
    od_tho.append(o_d_tho)
    acc_tho.append(acc_tho_)

    fpr_0_tho.append(fpr_0_tho_)
    tpr_0_tho.append(tpr_0_tho_)
    fpr_1_tho.append(fpr_1_tho_)
    tpr_1_tho.append(tpr_1_tho_)
    d_i_tho.append(d_i_tho_)
time_elapsed_tho = datetime.now() - start_time



c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


performance for whole ds:subexperiment - baseline THO model:0
conf matrix:
  [[498  48]
 [ 75 179]] 
accuracy:  0.84625 precision:  0.788546255506608 recall:  0.7047244094488189 fpr:  0.08791208791208792 tpr:  0.7047244094488189 fn+fp 123
performance for 0 sub-group:
conf matrix:
  [[235  19]
 [ 54  97]] 
accuracy:  0.8197530864197531 precision:  0.8362068965517241 recall:  0.6423841059602649 fpr:  0.07480314960629922 tpr:  0.6423841059602649 fn+fp 73
performance for 1 sub-group:
conf matrix:
  [[263  29]
 [ 21  82]] 
accuracy:  0.8734177215189873 precision:  0.7387387387387387 recall:  0.7961165048543689 fpr:  0.09931506849315068 tpr:  0.7961165048543689 fn+fp 50
bias metrics:
avarage absolute odds difference: (close to 0) 0.08912215889047775


c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


performance for whole ds:subexperiment - baseline THO model:1
conf matrix:
  [[476  70]
 [ 62 192]] 
accuracy:  0.835 precision:  0.732824427480916 recall:  0.7559055118110236 fpr:  0.1282051282051282 tpr:  0.7559055118110236 fn+fp 132
performance for 0 sub-group:
conf matrix:
  [[226  42]
 [ 26  86]] 
accuracy:  0.8210526315789474 precision:  0.671875 recall:  0.7678571428571429 fpr:  0.15671641791044777 tpr:  0.7678571428571429 fn+fp 68
performance for 1 sub-group:
conf matrix:
  [[250  28]
 [ 36 106]] 
accuracy:  0.8476190476190476 precision:  0.7910447761194029 recall:  0.7464788732394366 fpr:  0.10071942446043165 tpr:  0.7464788732394366 fn+fp 64
bias metrics:
avarage absolute odds difference: (close to 0) 0.0386876315338612


c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


performance for whole ds:subexperiment - baseline THO model:2
conf matrix:
  [[467  80]
 [ 62 191]] 
accuracy:  0.8225 precision:  0.7047970479704797 recall:  0.7549407114624506 fpr:  0.14625228519195613 tpr:  0.7549407114624506 fn+fp 142
performance for 0 sub-group:
conf matrix:
  [[197  55]
 [ 18  85]] 
accuracy:  0.7943661971830986 precision:  0.6071428571428571 recall:  0.8252427184466019 fpr:  0.21825396825396826 tpr:  0.8252427184466019 fn+fp 73
performance for 1 sub-group:
conf matrix:
  [[270  25]
 [ 44 106]] 
accuracy:  0.8449438202247191 precision:  0.8091603053435115 recall:  0.7066666666666667 fpr:  0.0847457627118644 tpr:  0.7066666666666667 fn+fp 69
bias metrics:
avarage absolute odds difference: (close to 0) 0.12604212866101955


c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


performance for whole ds:subexperiment - baseline THO model:3
conf matrix:
  [[487  60]
 [ 70 183]] 
accuracy:  0.8375 precision:  0.7530864197530864 recall:  0.7233201581027668 fpr:  0.10968921389396709 tpr:  0.7233201581027668 fn+fp 130
performance for 0 sub-group:
conf matrix:
  [[236  43]
 [ 17  87]] 
accuracy:  0.8433420365535248 precision:  0.6692307692307692 recall:  0.8365384615384616 fpr:  0.15412186379928317 tpr:  0.8365384615384616 fn+fp 60
performance for 1 sub-group:
conf matrix:
  [[251  17]
 [ 53  96]] 
accuracy:  0.8321342925659473 precision:  0.8495575221238938 recall:  0.6442953020134228 fpr:  0.06343283582089553 tpr:  0.6442953020134228 fn+fp 70
bias metrics:
avarage absolute odds difference: (close to 0) 0.1414660937517132
performance for whole ds:subexperiment - baseline THO model:4
conf matrix:
  [[493  54]
 [ 71 182]] 
accuracy:  0.84375 precision:  0.7711864406779662 recall:  0.7193675889328063 fpr:  0.09872029250457039 tpr:  0.7193675889328063 fn+fp 125
perform

c:\Users\seffi\anaconda3\envs\kats\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## Bias mitigation using FairTTA

In [29]:
experiment = 'baseline without TTA'
run = wandb.init(project='BiasGuardRF1', reinit = True)
print(experiment)
print('equal opportunity for tpr: ', np.mean(op), np.var(op))
print('equal opportunity for fpr: ',np.mean(op_f), np.var(op_f))
print('avarage absolute odds difference: ', np.mean(od), np.var(od))
print('accuracy: ', np.mean(acc), np.var(acc))
wandb.log({'dataset' : DATASET_NAME, 'model' : EXP, 'experiment' : experiment,
            'accuracy' : np.mean(acc), 'acc_var': np.var(acc),'dTPR' : np.mean(op), 'dFPR':np.mean(op_f),
            'avarge odd' : np.mean(od),'odd_var': np.var(od), 'DI':np.mean(d_i), 'DI_var': np.var(d_i), 'flips': None,'time' : str(time_elapsed_baseline) }, step=0)
wandb.finish()

baseline without TTA
equal opportunity for tpr:  0.11248675931953811 0.0018291896375059896
equal opportunity for fpr:  0.042158193241339195 0.0007641435047533319
avarage absolute odds difference:  0.07732247628043865 0.0008744459883223454
accuracy:  0.8532500000000001 2.7874999999999993e-05


DI,▁
DI_var,▁
acc_var,▁
accuracy,▁
avarge odd,▁
dFPR,▁
dTPR,▁
odd_var,▁
DI,1.11089
DI_var,0.18287
acc_var,3e-05


In [30]:
experiment = 'baseline with ThresholdOptimizer'
run = wandb.init(project='BiasGuardRF1', reinit=True)
print(experiment)
print('tpr0: ', np.mean(tpr_0_tho) , ' tpr1: ', np.mean(tpr_1_tho), ' delta: ', abs(np.mean(tpr_0_tho) - np.mean(tpr_1_tho)))
print('fpr0: ', np.mean(fpr_0_tho), 'fpr1: ', np.mean(fpr_1_tho), ' delta: ', abs(np.mean(fpr_0_tho) - np.mean(fpr_1_tho)))
print('equal opportunity for tpr: ',np.mean(op_tho))
print('equal opportunity for fpr: ',np.mean(op_f_tho))
print('avarage absolute odds difference: ',np.mean(od_tho))
print('accuracy: ', np.mean(acc_tho))
print('Time: ',time_elapsed_tho)
wandb.log({'dataset' : DATASET_NAME, 'model' : EXP, 'experiment' : experiment,
            'accuracy' : np.mean(acc_tho), 'acc_var':np.var(acc_tho) ,'dTPR' : np.mean(op_tho), 'dFPR':np.mean(op_f_tho),
            'avarge odd' : np.mean(od_tho),'odd_var': np.var(od_tho), 'DI':np.mean(d_i_tho), 'DI_var': np.var(d_i_tho), 'flips': flips, 'time' : str(time_elapsed_tho) }, step=0)
wandb.finish()

baseline with ThresholdOptimizer
tpr0:  0.7536449920896082  tpr1:  0.7302904167232  delta:  0.02335457536640817
fpr0:  0.13425425721896422 fpr1:  0.09605771263689108  delta:  0.03819654458207314
equal opportunity for tpr:  0.10952441700266399
equal opportunity for fpr:  0.07388114620612991
avarage absolute odds difference:  0.09170278160439696
accuracy:  0.8370000000000001
Time:  0:00:01.274879


DI,▁
DI_var,▁
acc_var,▁
accuracy,▁
avarge odd,▁
dFPR,▁
dTPR,▁
flips,▁
odd_var,▁
DI,0.89738
DI_var,0.01112


CTGAN

In [31]:
from sdv.sampling import Condition

In [32]:
if DATASET_NAME == 'ADULT_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_adult_sex.pkl')
elif DATASET_NAME == 'COMPAS_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_compas_sex.pkl')
elif DATASET_NAME == 'CREDIT_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_credit_sex.pkl')
elif DATASET_NAME == 'RECRUIT_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_recruit_sex.pkl')
elif DATASET_NAME == 'DIABETES_AGE':
    ctgan = CTGANSynthesizer.load('ctgan_diabetes_age.pkl')
elif DATASET_NAME == 'BANK_AGE':
    ctgan = CTGANSynthesizer.load('ctgan_bank_age.pkl')
elif DATASET_NAME == 'DEFAULT_CREDIT_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_default_credit_sex.pkl')
elif DATASET_NAME == 'NURSERY_OCC':
    ctgan = CTGANSynthesizer.load('ctgan_nursery_occ.pkl')
elif DATASET_NAME == 'COMPAS_RACE':
    ctgan = CTGANSynthesizer.load('ctgan_compas_race.pkl')
elif DATASET_NAME == 'ADULT_RACE':
    ctgan = CTGANSynthesizer.load('ctgan_adult_race.pkl')
elif DATASET_NAME == 'MIMIC_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_mimic_sex.pkl')
elif DATASET_NAME == 'MATERNAL_AGE':
    ctgan = CTGANSynthesizer.load('ctgan_maternal_age.pkl')
elif DATASET_NAME == 'MESO_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_meso_sex.pkl')
elif DATASET_NAME == 'SURGICAL_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_surgical_sex.pkl')
elif DATASET_NAME == 'SURGICAL_AGE':
    ctgan = CTGANSynthesizer.load('ctgan_surgical_age.pkl')
elif DATASET_NAME == 'DUTCH_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_dutch_sex.pkl')
elif DATASET_NAME == 'LAW_SEX':
    ctgan = CTGANSynthesizer.load('ctgan_law_sex.pkl')
synthetic_data = ctgan.sample(data.shape[0]*CTGAN_FACTOR)
synthetic_data.head(10)
condition_0 = Condition({PRIV_FEATURE: 0}, num_rows=data.shape[0])
condition_1 = Condition({PRIV_FEATURE: 1}, num_rows=data.shape[0])
synthetic_data_0 = ctgan.sample_from_conditions(conditions=[condition_0])
synthetic_data_1 = ctgan.sample_from_conditions(conditions=[condition_1])

Sampling conditions: 100%|██████████| 4000/4000 [00:00<00:00, 6206.20it/s]


In [33]:
synthetic_data =  encoder.transform(synthetic_data)
synthetic_data_0 =  encoder.transform(synthetic_data_0)
synthetic_data_1 =  encoder.transform(synthetic_data_1)

random forest TTA

# CTGAN TTA

In [34]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5)

In [35]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import Normalizer
synthetic_data = synthetic_data.dropna()
transformer = Normalizer().fit(synthetic_data.values)

In [36]:
svd_data = svd.fit_transform(transformer.transform(data.values))

In [37]:
svd_synthdata = svd.transform(transformer.transform(synthetic_data.values))

In [38]:
nbrs = NearestNeighbors(n_neighbors=2,metric='cosine').fit(synthetic_data.values)

In [39]:
synthetic_data_0 = synthetic_data_0.dropna()
synthetic_data_1 = synthetic_data_1.dropna()
svd_synthdata_0 = svd.transform(transformer.transform(synthetic_data_0.values))
svd_synthdata_1 = svd.transform(transformer.transform(synthetic_data_1.values))

In [40]:
nbrs_0 = NearestNeighbors(n_neighbors=1,metric='cosine',n_jobs=-1).fit(svd_synthdata_0)
nbrs_1 = NearestNeighbors(n_neighbors=1,metric='cosine',n_jobs=-1).fit(svd_synthdata_1)

In [41]:
pd.DataFrame(data.iloc[test].iloc[i].values.reshape(1,-1),columns=data.columns)

,gender,age,nationality,sport,ind-university_grade,ind-debateclub,ind-programming_exp,ind-international_exp,ind-entrepeneur_exp,ind-languages,ind-exact_study,ind-degree,company
0,0.0,27.0,0.308756,0.353206,60.0,0.467588,0.323162,0.291976,0.461454,2.0,0.33597,0.204637,0.424


In [42]:
import numpy as np

def calc_bias(
    X,
    y,
    priv_feature_idx,
    y_pred,
    experiment_text
):
    """Calculate bias metrics with purely NumPy-based logic."""
    # Ensure all inputs are NumPy arrays
    X = np.asarray(X)
    y = np.asarray(y)
    y_pred = np.asarray(y_pred)

    print("performance for whole ds:" + experiment_text)
    fpr, tpr, acc = print_metrics(y, y_pred)

    # Unique categories in the privileged feature column
    feature_categories = np.unique(X[:, priv_feature_idx])
    # e.g. cat0=0, cat1=1
    cat0, cat1 = feature_categories[0], feature_categories[1]

    # Boolean masks for each subgroup
    mask0 = (X[:, priv_feature_idx] == cat0)
    mask1 = (X[:, priv_feature_idx] == cat1)

    # Slice out each group
    y0 = y[mask0]
    y_pred0 = y_pred[mask0]
    y1 = y[mask1]
    y_pred1 = y_pred[mask1]

    print("performance for 0 sub-group:")
    fpr_0, tpr_0, acc_0 = print_metrics(y0, y_pred0)

    print("performance for 1 sub-group:")
    fpr_1, tpr_1, acc_1 = print_metrics(y1, y_pred1)

    op_diff = opportunity_diff_tpr(tpr_0, tpr_1)
    op_diff_fpr = opportunity_diff_fpr(fpr_0, fpr_1)
    od_diff = odds_diff(tpr_0, tpr_1, fpr_0, fpr_1)

    print("bias metrics:")
    print("average absolute odds difference: (close to 0)", od_diff)

    # disparate_impact must handle two 1D arrays
    di = disparate_impact(y_pred0.astype(int), y_pred1.astype(int))

    return (op_diff, op_diff_fpr, od_diff, acc, fpr_0, tpr_0, fpr_1, tpr_1, di)

In [43]:
import numpy as np

# Convert dataframes/series to NumPy arrays
data_array = data.to_numpy()
y_array = y.to_numpy()

synthetic_data_0_array = synthetic_data_0.to_numpy()
synthetic_data_1_array = synthetic_data_1.to_numpy()
svd_synthdata_0_array = svd_synthdata_0
svd_synthdata_1_array = svd_synthdata_1

In [44]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from datetime import datetime

fprs, tprs, scores = [], [], []
op_tta4, op_f_tta4, od_tta4, acc_tta4, d_i_tta4 = [], [], [], [], []
swaps_count = 0
start_time = datetime.now()

for fold, (train_idx, test_idx) in enumerate(cv.split(data_array, y_array)):
    # Fit on the training slice
    clf.fit(data_array[train_idx, :], y_array[train_idx])

    y_pred_tta = []
    for i in range(len(test_idx)):

        # Get the original row (shape = (n_features,))
        row = data_array[test_idx[i], :]

        # Expand to 2D for predict_proba (shape = (1, n_features))
        tta_ds = np.expand_dims(row, axis=0)

        # Quick check on predicted probability
        prob_tta_ds = np.mean(clf.predict_proba(tta_ds)[:, 1])
        y_pred_temp = round(prob_tta_ds)

        # Create “opposite” sample
        sample_opposite = row.copy()
        sample_opposite[PROT_ATTR_IDX] = abs(1 - sample_opposite[PROT_ATTR_IDX])

        prob_opposite = np.mean(
            clf.predict_proba(sample_opposite.reshape(1, -1))[:, 1]
        )

        # If the difference in predicted outcomes is 1 when rounded
        if abs(round(prob_tta_ds) - round(prob_opposite)) == 1:
            # If the privileged feature is 1
            if row[PROT_ATTR_IDX] == 1:
                # Use nearest neighbors from "0" group
                ind = nbrs_0.kneighbors(
                    svd_synthdata_0_array[i].reshape(1, -1),
                    n_neighbors=N_AUG,
                    return_distance=False
                )
                for j in range(N_AUG):
                    new_row = synthetic_data_0_array[ind[0][j], :]  # shape = (n_features,)
                    # Concatenate a (1, n_features) array
                    tta_ds = np.concatenate((tta_ds, new_row[np.newaxis, :]), axis=0)
            else:
                # Use nearest neighbors from "1" group
                ind = nbrs_1.kneighbors(
                    svd_synthdata_1_array[i].reshape(1, -1),
                    n_neighbors=N_AUG,
                    return_distance=False
                )
                for j in range(N_AUG):
                    new_row = synthetic_data_1_array[ind[0][j], :]
                    tta_ds = np.concatenate((tta_ds, new_row[np.newaxis, :]), axis=0)

            # Add duplicates of the original row (N_AUG - 1) times
            for _ in range(N_AUG - 1):
                tta_ds = np.concatenate((tta_ds, row[np.newaxis, :]), axis=0)

        # Final TTA prediction
        tta_prob_mean = np.mean(clf.predict_proba(tta_ds)[:, 1])
        y_pred_tta.append(round(tta_prob_mean))

    o_p, o_p_fpr, o_d, acc, fpr_0, tpr_0, fpr_1, tpr_1, d_i = calc_bias(
        data_array[test_idx, :],
        y_array[test_idx],
        PROT_ATTR_IDX,
        y_pred_tta,
        f"subexperiment - CTGAN TTA fold:{fold}"
    )
    op_tta4.append(o_p)
    op_f_tta4.append(o_p_fpr)
    od_tta4.append(o_d)
    acc_tta4.append(acc)
    d_i_tta4.append(d_i)

time_elapsed_ctgan = datetime.now() - start_time
print("Time elapsed for CTGAN TTA:", time_elapsed_ctgan)

performance for whole ds:subexperiment - CTGAN TTA fold:0
conf matrix:
  [[514  32]
 [ 83 171]] 
accuracy:  0.85625 precision:  0.8423645320197044 recall:  0.6732283464566929 fpr:  0.05860805860805861 tpr:  0.6732283464566929 fn+fp 115
performance for 0 sub-group:
conf matrix:
  [[230  24]
 [ 48 103]] 
accuracy:  0.8222222222222222 precision:  0.8110236220472441 recall:  0.6821192052980133 fpr:  0.09448818897637795 tpr:  0.6821192052980133 fn+fp 72
performance for 1 sub-group:
conf matrix:
  [[284   8]
 [ 35  68]] 
accuracy:  0.8911392405063291 precision:  0.8947368421052632 recall:  0.6601941747572816 fpr:  0.0273972602739726 tpr:  0.6601941747572816 fn+fp 43
bias metrics:
average absolute odds difference: (close to 0) 0.04450797962156852
performance for whole ds:subexperiment - CTGAN TTA fold:1
conf matrix:
  [[504  42]
 [ 76 178]] 
accuracy:  0.8525 precision:  0.8090909090909091 recall:  0.7007874015748031 fpr:  0.07692307692307693 tpr:  0.7007874015748031 fn+fp 118
performance for

In [45]:
experiment = 'BiasGuard TTA'+str(N_AUG)
run = wandb.init(project='BiasGuardRF1', reinit=True)
print(experiment)
print('equal opportunity for tpr: ',np.mean(op_tta4))
print('equal opportunity for fpr: ',np.mean(op_f_tta4))
print('avarage absolute odds difference: ',np.mean(od_tta4))
print('accuracy: ', np.mean(acc_tta4))
print('time: ', time_elapsed_ctgan)
print('swaps count: ', swaps_count)
wandb.log({'dataset' : DATASET_NAME, 'model' : EXP, 'experiment' : experiment,
            'accuracy' : np.mean(acc_tta4),'acc_var': np.var(acc_tta4), 'dTPR' : np.mean(op_tta4), 'dFPR':np.mean(op_f_tta4),
            'avarge odd' : np.mean(od_tta4),'odd_var': np.var(od_tta4), 'DI':np.mean(d_i_tta4), 'DI_var': np.var(d_i_tta4), 'flips': swaps_count, 'time' : str(time_elapsed_ctgan) }, step=0)
wandb.finish()

BiasGuard TTA6
equal opportunity for tpr:  0.08279615220808301
equal opportunity for fpr:  0.03544727735789568
avarage absolute odds difference:  0.059121714782989346
accuracy:  0.85175
time:  0:00:06.306459
swaps count:  0


DI,▁
DI_var,▁
acc_var,▁
accuracy,▁
avarge odd,▁
dFPR,▁
dTPR,▁
flips,▁
odd_var,▁
DI,0.69215
DI_var,0.0028
